In [1]:
# -*- coding: utf-8 -*-

def delta_encode(numbers: list[int]) -> list[int]:
    """
    Chuyển một danh sách số nguyên đã sắp xếp thành mã hóa delta.

    Args:
        numbers: Danh sách các số nguyên đã sắp xếp tăng dần.

    Returns:
        Danh sách đã được mã hóa delta.
    """
    if not numbers:
        return []
    
    # Số đầu tiên giữ nguyên, các số sau là khoảng cách
    deltas = [numbers[0]]
    for i in range(1, len(numbers)):
        gap = numbers[i] - numbers[i-1]
        deltas.append(gap)
    return deltas

In [2]:
print("--- DEMO DELTA ENCODING ---")
doc_ids = [8, 23, 25, 40, 58, 61, 100, 102]
print(f"Danh sách gốc:     {doc_ids}")

delta_encoded = delta_encode(doc_ids)
print(f"Sau khi mã hóa Delta: {delta_encoded}")

--- DEMO DELTA ENCODING ---
Danh sách gốc:     [8, 23, 25, 40, 58, 61, 100, 102]
Sau khi mã hóa Delta: [8, 15, 2, 15, 18, 3, 39, 2]


In [3]:
def vbyte_encode(number: int) -> tuple[bytes, int]:
    """
    Nén một số nguyên bằng V-Byte và trả về số byte đã sử dụng.

    Args:
        number: Số nguyên không âm cần nén.

    Returns:
        Một tuple gồm (dữ liệu đã nén dưới dạng bytes, số byte đã dùng).
    """
    # Xử lý trường hợp đặc biệt số 0
    if number == 0:
        encoded_bytes = bytes([128])   # 10000000
        return encoded_bytes, len(encoded_bytes)

    byte_list = []
    # Lặp để tách số thành các khối 7-bit
    while number > 0:
        # Lấy 7 bit cuối cùng bằng bitmask    
        seven_bits = number & 0x7F
        byte_list.insert(0, seven_bits)
        
        # Dịch phải 7 bit để xử lý phần còn lại của số
        number >>= 7

    # Byte cuối cùng (bây giờ là phần tử cuối trong list) 
    # cần được set bit cờ (bit đầu tiên) thành 1.
    # 0x80 là 10000000, phép OR sẽ set bit đầu tiên thành 1.
    byte_list[-1] |= 0x80  #10000000

    # Chuyển danh sách các số thành đối tượng bytes
    encoded_bytes = bytes(byte_list)
    
    return encoded_bytes, len(encoded_bytes)

In [4]:
print("--- DEMO V-BYTE ENCODING ---")

# Danh sách các số để kiểm tra, bao gồm các giá trị biên
numbers_to_test = [0, 6, 127, 128, 130, 16383, 16384, 20000]

print(f"{'Số Gốc':<10} | {'Mã Hex':<20} | {'Số Byte Đã Dùng'}")
print(f"{'-'*10} | {'-'*20} | {'-'*15}")

for num in numbers_to_test:

    encoded_data, num_bytes = vbyte_encode(num)
    
    hex_representation = encoded_data.hex(' ')
    
    print(f"{num:<10} | {hex_representation:<20} | {num_bytes}")

--- DEMO V-BYTE ENCODING ---
Số Gốc     | Mã Hex               | Số Byte Đã Dùng
---------- | -------------------- | ---------------
0          | 80                   | 1
6          | 86                   | 1
127        | ff                   | 1
128        | 01 80                | 2
130        | 01 82                | 2
16383      | 7f ff                | 2
16384      | 01 00 80             | 3
20000      | 01 1c a0             | 3
